In [ ]:
import pandas as pd
import numpy as np
from mip import *
from pulp import *
import csv
from pulp import GLPK
import matplotlib.pyplot as plt
import math
import statistics
from pytictoc import TicToc
import time
from ttictoc import tic,toc
import requests
import json
import polyline
import folium

In [ ]:
# Supplier profile
Supplier_profile = pd.read_csv("monthly suppliers profile.csv")
display(Supplier_profile)

In [ ]:
# Suppliers profile

B = 17 # set of biomass
S = 77 # set of suppliers
T = 12 # set of time-step
Supplier_Profile=np.zeros([B,S,T])
for s in range(S):
    for b in range(B):
        for t in range(T):
                Supplier_Profile[b,s,t] = Supplier_profile.iloc[(17*s)+b,t+2]

In [ ]:
# Power plant profile
PowerPlant_profile = pd.read_csv("plants profile.csv")
PowerPlant_profile

In [ ]:
# Distance
Distance = pd.read_csv("distance.csv")
display(Distance)

In [ ]:
# Biomass data
Biomass_data = pd.read_csv("biomass.csv")
display(Biomass_data)

In [ ]:
# Latitude Longitude
LPtP = pd.read_csv("industrial area loc.csv")
display(LPtP)

In [ ]:
num_days_in_month = np.array([31,28,31,30,31,30,31,31,30,31,30,31])

# ==========================================================
# Optimize with the lowest additional biomass
# ==========================================================

In [ ]:
tic()

C_TF = 455542.86 # Fixed transportation cost per a truck
D_Eth = 4790000 #L/day
D_BElc = 3940 # MW (target at 3940 MW)
D_GElc = 387 # MW (target at 387 MW)
C_IF = 0 #THB/ton

# PuLP
# Set==========================================================================
B = 17 # set of biomass
S = 77 # set of suppliers
PQ123 = 215 # set of plant
PQ4 = 185
PQ5 = 27
T = 12 # set of time period
F = 77
M = 2 # new plants
# D = 77 #set of demand


prob = LpProblem(name = 'AdditionalBiomass', sense = LpMinimize)

# Variables ==============================================================

Xb123 = LpVariable.dicts("Amount_of_biomass_delivered_to_plant_withQ123",((b,s,p,t) for b in range(B) for s in range(S) for p in range (PQ123) for t in range(T)),lowBound=0, cat='Continuous')
Xb4 = LpVariable.dicts("Amount_of_biomass_delivered_to_plant_withQ4",((b,s,p,t) for b in range(B) for s in range(S) for p in range (PQ4) for t in range(T)),lowBound=0, cat='Continuous')
Xb5 = LpVariable.dicts("Amount_of_biomass_delivered_to_plant_withQ5",((b,s,p,t) for b in range(B) for s in range(S) for p in range (PQ5) for t in range(T)),lowBound=0, cat='Continuous')

Xp123=LpVariable.dicts("Amount_of_biomass_fed_into_process_withQ123",((b,p,t) for b in range(B) for p in range (PQ123) for t in range(T)),lowBound=0, cat='Continuous')
Xp4=LpVariable.dicts("Amount_of_biomass_fed_into_process_withQ4",((b,p,t) for b in range(B) for p in range (PQ4) for t in range(T)),lowBound=0, cat='Continuous')
Xp5=LpVariable.dicts("Amount_of_biomass_fed_into_process_withQ5",((b,p,t) for b in range(B) for p in range (PQ5) for t in range(T)),lowBound=0, cat='Continuous')

I123 = LpVariable.dicts("Inventory_withQ123",((b,p,t) for b in range(B) for p in range (PQ123) for t in range(T)),lowBound=0, cat='Continuous')
I4 = LpVariable.dicts("Inventory_withQ4",((b,p,t) for b in range(B) for p in range (PQ4) for t in range(T)),lowBound=0, cat='Continuous')
I5 = LpVariable.dicts("Inventory_withQ5",((b,p,t) for b in range(B) for p in range (PQ5) for t in range(T)),lowBound=0, cat='Continuous')
Max_I123 = LpVariable.dicts("Maximum Inventory Level_withQ123", ((p) for p in range (PQ123)), lowBound=0, cat='Continuous')
Max_I4 = LpVariable.dicts("Maximum Inventory Level_withQ4", ((p) for p in range (PQ4)), lowBound=0, cat='Continuous')
Max_I5 = LpVariable.dicts("Maximum Inventory Level_withQ5", ((p) for p in range (PQ5)), lowBound=0, cat='Continuous')

Badd = LpVariable.dicts("Additional_biomass",((b,s,t) for b in range(B) for s in range(S) for t in range(T)),lowBound=0, cat='Continuous')
Xbnew = LpVariable.dicts("Amount_of_biomass_delivered_to_new_plant",((b,s,m,t) for b in range(B) for s in range(S) for m in range(M) for t in range(T)),lowBound=0, cat='Continuous')
Xpnew = LpVariable.dicts("Amount_of_biomass_fed_into_process_at_new_plant",((b,m,t) for b in range(B) for m in range(M) for t in range(T)),lowBound=0, cat='Continuous')
Inew = LpVariable.dicts("Inventory_at_new_plant",((b,m,t) for b in range(B) for m in range(M) for t in range(T)),lowBound=0, cat='Continuous')
Max_Inew = LpVariable.dicts("Maximum Inventory Level_at_new_plant",((m) for m in range(M)), lowBound=0, cat='Continuous')
Elc_max= LpVariable.dicts("Max_Elc_prod_cap_new_plant",((m) for m in range(M)),lowBound=0, cat='Continuous')
# Ynew = LpVariable.dicts("Binary",((s,f) for s in range(S) for f in range(F)), cat='Binary')

# Objective ==========================================================================

prob+= lpSum(Badd[b,s,t] for b in range(B) for s in range(S) for t in range(T))


# Constraints==========================================================================


    
### Limitation of biomass
for b in range(B):
    for s in range(S):
        for t in range(T):
            prob+= lpSum(Xb123[b,s,p,t] for p in range(PQ123))+lpSum(Xb4[b,s,p,t] for p in range(PQ4))+lpSum(Xb5[b,s,p,t] for p in range(PQ5))+lpSum(Xbnew[b,s,m,t] for m in range(M)) <= Supplier_Profile[b,s,t]+Badd[b,s,t]
            
### Limitation of inventory
for p in range(PQ123):
    for t in range(T):
        prob+=lpSum(I123[b,p,t] for b in range(B))<= Max_I123[p]
for p in range(PQ4):
    for t in range(T):
        prob+=lpSum(I4[b,p,t] for b in range(B))<= Max_I4[p]
for p in range(PQ5):
    for t in range(T):
        prob+=lpSum(I5[b,p,t] for b in range(B))<= Max_I5[p]
###### Limitation of new inventory        
for t in range(T):
    for m in range(M):
        prob+=lpSum(Inew[b,m,t] for b in range(B))<= Max_Inew[m]

    
###  Inventory change
for b in range(B):
    for p in range(PQ123):
        for t in range(T):
            if t==0:
                prob += I123[b,p,t] == lpSum(Xb123[b,s,p,t] for s in range(S)) - Xp123[b,p,t]
            else:
                prob+= I123[b,p,t] == I123[b,p,t-1]+lpSum(Xb123[b,s,p,t] for s in range(S)) - Xp123[b,p,t]
                
for b in range(B):
    for p in range(PQ4):
        for t in range(T):
            if t==0:
                prob += I4[b,p,t] == lpSum(Xb4[b,s,p,t] for s in range(S)) - Xp4[b,p,t]
            else:
                prob+= I4[b,p,t] == I4[b,p,t-1]+lpSum(Xb4[b,s,p,t] for s in range(S)) - Xp4[b,p,t]
                
for b in range(B):
    for p in range(PQ5):
        for t in range(T):
            if t==0:
                prob += I5[b,p,t] == lpSum(Xb5[b,s,p,t] for s in range(S)) - Xp5[b,p,t]
            else:
                prob+= I5[b,p,t] == I5[b,p,t-1]+lpSum(Xb5[b,s,p,t] for s in range(S)) - Xp5[b,p,t]
                
###### New inventory change
for b in range(B):
    for t in range(T):
        for m in range(M):
            if t==0:
                prob += Inew[b,m,t] == lpSum(Xbnew[b,s,m,t] for s in range(S)) - Xpnew[b,m,t]
            else:
                prob+= Inew[b,m,t] == Inew[b,m,t-1]+lpSum(Xbnew[b,s,m,t] for s in range(S)) - Xpnew[b,m,t]
                
                
                
### Biomass fed into process cannot exceed inventory current level
for b in range(B):
    for p in range(PQ123):
        for t in range(T):
            if t==0:
                prob+= Xp123[b,p,t] <= lpSum(Xb123[b,s,p,t] for s in range(S)) 
            else:
                prob+= Xp123[b,p,t] <= I123[b,p,t-1]+ lpSum(Xb123[b,s,p,t] for s in range(S)) 
                
for b in range(B):
    for p in range(PQ4):
        for t in range(T):
            if t==0:
                prob+= Xp4[b,p,t] <= lpSum(Xb4[b,s,p,t] for s in range(S)) 
            else:
                prob+= Xp4[b,p,t] <= I4[b,p,t-1]+ lpSum(Xb4[b,s,p,t] for s in range(S))
                
for b in range(B):
    for p in range(PQ5):
        for t in range(T):
            if t==0:
                prob+= Xp5[b,p,t] <= lpSum(Xb5[b,s,p,t] for s in range(S)) 
            else:
                prob+= Xp5[b,p,t] <= I5[b,p,t-1]+ lpSum(Xb5[b,s,p,t] for s in range(S))
                
###### New biomass fed to process               
for b in range(B):
    for t in range(T):
        for m in range(M):
            if t==0:
                prob+= Xpnew[b,m,t] <= lpSum(Xbnew[b,s,m,t] for s in range(S)) 
            else:
                prob+= Xpnew[b,m,t] <= Inew[b,m,t-1]+ lpSum(Xbnew[b,s,m,t] for s in range(S))
                
                
                
## Limitation of electricity and bioethanol produced
for p in range(PQ123):
    for t in range(T):
        prob+= PowerPlant_profile['Electricity coef (MWh/MJ)'][p]*lpSum(Xp123[b,p,t]*Biomass_data['Heat Capacity (MJ/ton)'][b] for b in range(B)) <= PowerPlant_profile['Maximum electricity production capacity (MW)'][p]*24*num_days_in_month[t]*PowerPlant_profile['maximum plant factor'][p]

for p in range(PQ4):
    for t in range(T):
        prob+= PowerPlant_profile['Electricity coef (MWh/MJ)'][p+PQ123]*lpSum(Xp4[b,p,t]*Biomass_data['Methane heat eq (MJ/ton)'][b] for b in range(B)) <= PowerPlant_profile['Maximum electricity production capacity (MW)'][p+PQ123]*24*num_days_in_month[t]*PowerPlant_profile['maximum plant factor'][p]
        
for p in range(PQ5):
    for t in range(T):
        prob+= lpSum(Xp5[b,p,t]*Biomass_data['Bioethanol coef (L/ton)'][b] for b in range(B)) <= PowerPlant_profile['Maximum bioethanol production capacity (L/day)'][p+PQ123+PQ4]*num_days_in_month[t]*PowerPlant_profile['maximum plant factor'][p+PQ123+PQ4]
###### New electricity produced
for t in range(T):
    for m in range(M):
        if m == 0:
            prob+= 0.000125*lpSum(Xpnew[b,m,t]*Biomass_data['Heat Capacity (MJ/ton)'][b] for b in range(B)) <= Elc_max[m]*24*num_days_in_month[t]*0.85
        if m == 1:
            prob += 0.0000416*lpSum(Xpnew[b,m,t]*Biomass_data['Methane heat eq (MJ/ton)'][b] for b in range(B)) <=  Elc_max[m]*24*num_days_in_month[t]*0.85


### Biomass to Electricity demand constraint
for t in range(T):
    prob+= lpSum(PowerPlant_profile['Electricity coef (MWh/MJ)'][p]*lpSum(Xp123[b,p,t]*Biomass_data['Heat Capacity (MJ/ton)'][b] for b in range(B)) for p in range(PQ123))+(0.000125*lpSum(Xpnew[b,0,t]*Biomass_data['Heat Capacity (MJ/ton)'][b] for b in range(B))) == D_BElc*24*num_days_in_month[t]


### Biogas to Electricity demand constraint
for t in range(T):
    prob+= lpSum(PowerPlant_profile['Electricity coef (MWh/MJ)'][p+PQ123]*lpSum(Xp4[b,p,t]*Biomass_data['Methane heat eq (MJ/ton)'][b] for b in range(B)) for p in range(PQ4))+(0.0000416*lpSum(Xpnew[b,1,t]*Biomass_data['Methane heat eq (MJ/ton)'][b] for b in range(B))) == D_GElc*24*num_days_in_month[t]
### Ethanol demand constraint
for t in range(T):
    prob+= lpSum(Xp5[b,p,t]*Biomass_data['Bioethanol coef (L/ton)'][b] for b in range(B) for p in range(PQ5)) == D_Eth*num_days_in_month[t]

    
### Seasonal and local biomass constraints
for b in range(B):
    for s in range(S):
        for t in range(T):
            if s == 0:
                if (b != 0) and (b != 1):
                    prob += Badd[b,s,t] == 0
                    
            elif s==1:
                if b<=1:
                    if (t<=1) and (6<=t<=8):
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    if (t != 0) and (t != 1) and (t !=11):
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    prob+= Badd[b,s,t] ==0
                    
            elif s == 2:
                if 2<=b<=4:
                    if (t != 0) and (t != 1) and (t !=11):
                        prob+= Badd[b,s,t] ==0
                elif 5<=b<=16:
                    prob+= Badd[b,s,t] ==0
                    
            elif s ==3:
                if b<=1:
                    if t<=1 and (5<=t<=8):
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    if (t != 0) and (t != 1) and (t !=11):
                        prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    prob+= Badd[b,s,t] ==0
                    
            elif s ==4:
                if b<=1:
                    if t<=1 and (5<=t<=8):
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    if (t != 0) and (t != 1) and (t !=11):
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    prob+= Badd[b,s,t] ==0
                    
            elif s==5:
                if 2<=b<=4:
                    if (t != 0) and (t != 1) and (t !=11):
                        prob+= Badd[b,s,t] ==0
                        
            elif s ==6:
                if 2<=b<=4:
                    if (t != 0) and (t != 1) and (t !=11):
                        prob+= Badd[b,s,t] ==0
                if (t != 0) and (t != 1) and (t !=11):
                    if 6<=t<=8:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    if (t == 5) and (t ==7):
                        prob+= Badd[b,s,t] ==0
                        
            elif s == 7:
                if b<=1:
                    if (t<=1) and (1<=t<=8):
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    if (t != 0) and (t != 1) and (t !=11):
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    if (t == 5) and (t==7):
                        prob+= Badd[b,s,t] ==0
                        
            elif s ==8:
                if b<=1:
                    if t != 9 and t !=10 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    if (t != 0) and (t != 1) and (t !=11):
                        prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    if 5<=t<=8:
                        prob+= Badd[b,s,t] ==0
                        
            elif s == 9:
                if b<=1:
                    if t != 8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    prob += Badd[b,s,t] ==0
                elif 7<=b<=10:
                    if 5<=t<=8:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    prob+= Badd[b,s,t] ==0
                    
            elif s == 10:
                if b<=1:
                    if t == 1 and t ==8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    if 5<=t<=8:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    prob+= Badd[b,s,t] ==0
                    
            elif s == 11:
                if 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0 
                        
            elif s == 12:
                if b <=1:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    prob+= Badd[b,s,t] ==0
                    
            elif s==13:
                if b <=1:
                    if t == 0 and t ==6 and t ==7 and t ==8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    prob+= Badd[b,s,t] ==0
                    
            elif s ==14:
                if 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    if t ==8:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    if t ==5:
                        prob+= Badd[b,s,t] ==0
                        
            elif s==15:
                if b <=1:
                    if t == 8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                        
            elif s == 16:
                if b<=1:
                    if t == 0 and t ==1 and t ==7 and t ==8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    prob+= Badd[b,s,t] ==0
                    
            elif s == 17:
                if b <=1:
                    if t != 0 and t !=1 and t !=10 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    prob+= Badd[b,s,t] ==0
                    
            elif s==18:
                if b <=1:
                    if t == 0 and t ==1 and t ==6 and t ==7 and t ==8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    if 6<=b<=9:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    prob+= Badd[b,s,t] ==0
                    
            elif s == 19:
                if b <=1:
                    if t == 0 and t ==1 and t ==8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    if t >=5:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    prob+= Badd[b,s,t] ==0
                    
            elif s ==20:
                if b <=1:
                    if t != 3 and t !=4 and t !=9 and t !=10 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    if 4<=t<=8:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    prob+= Badd[b,s,t] ==0
                    
            elif s==21:
                if 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    prob+= Badd[b,s,t] ==0
                    
                    
            elif s == 22:
                if b<=1:
                    if t != 5 and t !=9 and t !=10 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif b != 5 and b !=6:
                    prob+= Badd[b,s,t] ==0
                    
            elif s == 23:
                if b <=1:
                    if t == 0 and t ==1 and t ==8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    prob+= Badd[b,s,t] ==0
                    
            elif s ==24:
                if b <=1:
                    if t == 0 and t ==1 and t ==5 and t ==6 and t ==7 and t ==8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    prob+= Badd[b,s,t] ==0
                    
            elif s == 25:
                if 2<=b<=6:
                    prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    if t == 1 and t ==6 and t ==9 and t ==7 and t ==8:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    if t == 6 and t ==7:
                        prob+= Badd[b,s,t] ==0
                        
            elif s == 26:
                if 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    prob+= Badd[b,s,t] ==0 
                    
            elif s == 27:
                if b<=1:
                    if t == 0 and t ==1 and t ==8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    if 5<=b<=9:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    prob+= Badd[b,s,t] ==0
                    
            elif s==28:
                if b <=1:
                    if t ==  8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    if t == 5:
                        prob+= Badd[b,s,t] ==0
                        
            elif s == 29:
                if 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    if 5<=t<=9:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    if t == 5 and t == 7:
                        prob+= Badd[b,s,t] ==0
                        
            elif s==30:
                if b <=1:
                    if 4<=b<=8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    prob+= Badd[b,s,t] ==0
                    
            elif s == 31:
                if b <=1:
                    if t !=  3 and t != 4 and t != 9 and t != 10 and t != 11:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    if 6<=t<=10:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    prob+= Badd[b,s,t] ==0
                    
            elif s ==32:
                if b <=1:
                    if t !=  1 and t != 2 and t != 3:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    prob+= Badd[b,s,t] ==0 
                    
            elif s==33:
                if b<=1:
                    if t !=  2 and t !=3 and t !=4 and t !=9 and t !=10 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    if t ==8: 
                        prob+= Badd[b,s,t] ==0 
                elif 14<=b<=16:
                    prob+= Badd[b,s,t] ==0
                    
            elif s==34:
                if b <=1:
                    if t ==  0 and t == 6 and t == 7 and t == 8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    if 5<= t <=8: 
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    prob+= Badd[b,s,t] ==0
                    
            elif s==35:
                if 2<= b<=13:
                    prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    if t ==7:
                        prob+= Badd[b,s,t] ==0
            
            elif s==36:
                if 2<=b<=4:
                    prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    if t ==7:
                        prob+= Badd[b,s,t] ==0
                        
            elif s==37:
                if b <=1:
                    if t !=  0 and t != 1 and t != 2 and t != 3:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    prob+= Badd[b,s,t] ==0
                    
            elif s ==38:
                if b <=1:
                    if t !=  0 and t != 1 and t != 10 and t != 11:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    prob+= Badd[b,s,t] ==0
                    
            elif s==39:
                if b<=1:
                    if t !=  0 and t !=1 and t !=2 and t !=3  and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    prob+= Badd[b,s,t] ==0
                    
            elif s == 40:
                if b <=1:
                    if t == 1 and t ==  6 and t == 8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    if t != 1 and t !=2 and t !=3 and t !=4:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    if t ==7:
                        prob+= Badd[b,s,t] ==0
                        
            elif s==41:
                if 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    if 6<=b<=8:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    if t ==7:
                        prob+= Badd[b,s,t] ==0
                        
            elif s==42:
                if 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    if t ==8:
                        prob+= Badd[b,s,t] ==0
                        
            elif s==43:
                if 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    if 4<=t<=9:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    if t ==5 and t ==7:
                        prob+= Badd[b,s,t] ==0
                        
            elif s==44:
                if 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    if 6<=t<=8:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    if t ==5 and t ==7:
                        prob+= Badd[b,s,t] ==0
                        
            elif s== 45:
                if 2<=b<=6:
                    prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    if t != 10:
                        prob+= Badd[b,s,t] ==0
                elif 11<=b<=13:
                    prob+= Badd[b,s,t] ==0
                    
            elif s==46:
                if b<=1:
                    if t == 0 and t ==1 and t ==5 and t ==6 and t ==7 and t ==8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif b == 5 and b == 6:
                    if t == 5:
                        prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    if t != 0 and t != 1 and t != 2 and t != 10 and t != 11:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    prob+= Badd[b,s,t] ==0
                    
            elif s==47:
                if b <=1:
                    if t != 0 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 2 <=b<= 10:
                    prob+= Badd[b,s,t] ==0
                    
            elif s==48:
                if 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif b == 6 and b ==7:
                    if t == 0 and t ==1 and t ==4 and t ==5:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    if t ==5 and t ==7:
                        prob+= Badd[b,s,t] ==0
                        
            elif s == 49:
                if b == 0 and b ==1:
                    if t == 1 and t ==8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    if 4<=t<=9:
                        prob+= Badd[b,s,t] ==0
                        
            elif s ==50:
                if b <=1:
                    if t != 0 and t !=1 and t !=10 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=10:
                    prob+= Badd[b,s,t] ==0
                    
            elif s==51:
                if 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                        
            elif s==52:
                if b<=1:
                    if t == 1 and t ==8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                        
            elif s==53:
                if b <=1:
                    if t == 0 and t ==1 and t ==7 and t ==8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    prob+= Badd[b,s,t] ==0
                    
            elif s==54:
                if b <=1:
                    if t == 1 and t ==6 and t ==7 and t ==8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    prob+= Badd[b,s,t] ==0
                    
            elif s==55:
                if b <=1:
                    if t == 0 and t ==1 and t ==5 and t ==6 and t ==7 and t ==8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    if t ==5:
                        prob+= Badd[b,s,t] ==0
                
            elif s==56:
                if 2<= b<=10:
                    prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    if t ==5:
                        prob+= Badd[b,s,t] ==0
                        
            elif s==57:
                if 2<= b<=13:
                    prob+= Badd[b,s,t] ==0
                    
            elif s==58:
                if b <=1:
                    if t != 0 and t != 4 and t != 5 and t != 6 and t != 10 and t != 11:
                        prob+= Badd[b,s,t] ==0
                elif 2<= b<=13:
                    prob+= Badd[b,s,t] ==0
                    
            elif s==59:
                if 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    if 5<=t<=9:
                        prob+= Badd[b,s,t] ==0
                elif 14 <=b<= 16:
                    if t ==5 and t ==7:
                        prob+= Badd[b,s,t] ==0
                
            elif s==60:
                if b <=1:
                    if t != 0 and t !=1  and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    prob+= Badd[b,s,t] ==0
                    
            elif s==61:
                if 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif (5<=b<=10) and (14<=b<=16):
                    prob+= Badd[b,s,t] ==0
                    
            elif s==62:
                if b <=1:
                    if t == 0 and t ==1 and t == 6 and t ==7 and t ==8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    if t ==5 and t ==7:
                        prob+= Badd[b,s,t] ==0
                        
            elif s==63:
                if b <=1:
                    if 4 <= t <= 10:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    prob+= Badd[b,s,t] ==0
                    
            elif s==64:
                if 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    if t ==8:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    prob+= Badd[b,s,t] ==0
                    
            elif s==65:
                if 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    if 5<=t<=8:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    if t ==5 and t ==7:
                        prob+= Badd[b,s,t] ==0
                        
            elif s==66:
                if b <=1:
                    if (2<=t<=6) and t==8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    prob+= Badd[b,s,t] ==0
                    
            elif s==67:
                if b <=1:
                    if t == 0 and t ==1 and t ==5 and t ==6 and t ==8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                        
            elif s==68:
                if b <=1:
                    if t == 1 and t ==7 and t ==8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    if t ==5 and t ==7:
                        prob+= Badd[b,s,t] ==0
                        
            elif s==69:
                if b <=1:
                    if 3<=t<=10:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    prob+= Badd[b,s,t] ==0
            
            elif s==70:
                if b<=1:
                    if t <=1 and 5<=t<=8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=10:
                    prob+= Badd[b,s,t] ==0
                    
            elif s==71:
                if b<=1:
                    if t <=1 and 6<=t<=8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    if t==5 and t==7:
                        prob+= Badd[b,s,t] ==0
                        
            elif s==72:
                if b<=1:
                    if t == 0 and 6<=t<=8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    if t==5 and t==7:
                        prob+= Badd[b,s,t] ==0
                        
            elif s==73:
                if 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    prob+= Badd[b,s,t] ==0
                    
            elif s==74:
                if 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    if 4<=t<=8:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    prob+= Badd[b,s,t] ==0
                    
            elif s==75:
                if b <=1:
                    if t != 1 and t !=2 and t !=3:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    prob+= Badd[b,s,t] ==0
                elif 7<=b<=10:
                    prob+= Badd[b,s,t] ==0
                    
            elif s==76:
                if b<=1:
                    if t == 0 and 6<=t<=8:
                        prob+= Badd[b,s,t] ==0
                elif 2<=b<=4:
                    if t != 0 and t !=1 and t !=11:
                        prob+= Badd[b,s,t] ==0
                elif 14<=b<=16:
                    prob+= Badd[b,s,t] ==0
                    
            elif 2<=b<=4:
                if t != 0 and t !=1 and t !=11:
                    prob+= Badd[b,s,t] ==0
                
                
                
# Solve==========================================================================
                                                                                
status=prob.solve() 

elapsed = toc()
# print("Optimization status: ",LpStatus[status])
print('Elapsed time:',elapsed)

In [ ]:
# Optimal value ====================================================================
print("Optimization status: ",LpStatus[status])
print("Optimal Objective value: ", value(prob.objective), "tons \n \n")

In [ ]:
# show additional biomass
Optimal_Badd = np.zeros([B,S,T])
for b in range(B):
    for s in range(S):
        for t in range(T):
            Optimal_Badd[b,s,t] = Badd[b,s,t].varValue
            
for s in range(S):
    print('Supplier:  ', s+1)
    pd.set_option('display.max_rows', None)
    DF = pd.DataFrame(Optimal_Badd[:,s,:])
    display(DF)